<a href="https://colab.research.google.com/github/brodyvachon/TennisMatchMLModel/blob/main/Tennis_Match_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv("merged_file_matches.csv", index_col=0)
players = pd.read_csv("atp_players.csv", index_col=0)
rankings = pd.read_csv("merged_file_rankings.csv")

In [ ]:
matches.head()

,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
tourney_id,,,,,,,,,,,,,,,,,,,,,
2000-301,Auckland,Hard,32,A,20000110,1,103163,1,NaN,Tommy Haas,...,55,39,29,17,4,7,11,1612,63,595
2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,Juan Balcells,...,32,25,18,12,3,6,211,157,49,723
2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,NaN,Alberto Martin,...,33,20,7,8,7,11,48,726,59,649
2000-301,Auckland,Hard,32,A,20000110,4,103507,7,NaN,Juan Carlos Ferrero,...,43,29,14,10,6,8,45,768,61,616
2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,Michael Sell,...,46,34,18,12,5,9,167,219,34,873


In [ ]:
players.head()

,name_first,name_last,hand,dob,ioc,height,wikidata_id
player_id,,,,,,,
100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581
100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049
100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752
100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554


In [ ]:
matches.shape

(24999, 48)

In [ ]:
matches.dtypes

,0
tourney_name,object
surface,object
draw_size,object
tourney_level,object
tourney_date,object
match_num,object
winner_id,object
winner_seed,object
winner_entry,object
winner_name,object


In [ ]:
matches["surface_code"] = matches["surface"].astype("category").cat.codes

In [ ]:
matches["player_hand_code"] = players["hand"].astype("category").cat.codes

In [ ]:
matches["player_dob"] = players["dob"]

In [ ]:
matches["player_ioc_code"] = players["ioc"].astype("category").cat.codes

In [ ]:
# Replace NaN values in 'winner_rank' with a suitable integer (e.g., -1) before conversion
import numpy as np

# Ensure 'winner_rank' column contains valid numeric data before conversion
# If the column contains string "winner_rank" replace it with -1
rankings['rank'] = pd.to_numeric(rankings['rank'], errors='coerce').fillna(-1)

matches["player_rank"] = rankings["rank"].astype(int)

In [ ]:
matches["ranking_date"] = rankings["ranking_date"]

In [ ]:
players["full_name"] = players["name_first"] + players["name_last"]

In [ ]:
players["full_name"]

,full_name
player_id,
100001,GardnarMulloy
100002,PanchoSegura
100003,FrankSedgman
100004,GiuseppeMerlo
100005,RichardGonzalez
...,...
212913,PietroRicci
212914,CoreyCraig
212915,AleksandarLjubojevic


In [ ]:
matches["winner_name"] = matches["winner_name"].str.replace(" ", "", regex=True)

In [ ]:
matches["winner_name"]

,winner_name
tourney_id,
2000-301,TommyHaas
2000-301,JuanBalcells
2000-301,AlbertoMartin
2000-301,JuanCarlosFerrero
2000-301,MichaelSell
...,...
2024-0425,TomasMachac
2024-0425,DusanLajovic
2024-0425,MatteoArnaldi


In [ ]:
matches["target"] = matches["winner_name"].isin(players["full_name"]).astype("int")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches["tourney_date"] < '2023-01-01']

In [ ]:
test = matches[matches["tourney_date"] > '2023-01-01']

In [ ]:
matches.dtypes

,0
tourney_name,object
surface,object
draw_size,object
tourney_level,object
tourney_date,object
match_num,object
winner_id,object
winner_seed,object
winner_entry,object
winner_name,object


In [ ]:
print(train.columns)

Index(['tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date',
       'match_num', 'winner_id', 'winner_seed', 'winner_entry', 'winner_name',
       'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age', 'loser_id',
       'loser_seed', 'loser_entry', 'loser_name', 'loser_hand', 'loser_ht',
       'loser_ioc', 'loser_age', 'score', 'best_of', 'round', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms',
       'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt', 'l_1stIn',
       'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'surface_code', 'player_hand_code', 'player_dob', 'player_ioc_code',
       'player_rank', 'ranking_date', 'target'],
      dtype='object')


In [ ]:
predictors = ["player_rank", "ranking_date", "player_dob", "player_ioc_code", "surface_code", "player_hand_code"]

In [ ]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = accuracy_score(test["target"], preds)

In [ ]:
acc

0.8555885262116716

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds)) #dataframe that combines our actual values and predicted values

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"]) #cross tabulation (shows when we predicted a 0 or 1 what actaully happenned)

predicted,1
actual,
0,292
1,1730


In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(test["target"], preds) #when we predicted a win, what percentage of the time did they actually win

0.8555885262116716

In [ ]:
grouped_matches = matches.groupby("winner_name") #create 1 dataframe for every player in our data

In [ ]:
group = grouped_matches.get_group("NovakDjokovic")

In [ ]:
# This function calculates rolling averages for specified columns over the last 5 matches
# in a player's match history, sorted by date. It first sorts the data chronologically,
# computes the rolling mean on the selected columns (excluding the current row), assigns
# these values to new columns, and removes rows that don't have enough prior matches for
# a complete 5-match average. This helps in analyzing a player’s recent form by using
# rolling averages for the desired stats.
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("tourney_date")
    # Convert relevant columns to numeric before calculating rolling mean
    for col in cols:
        group[col] = pd.to_numeric(group[col], errors='coerce')
    rolling_stats = group[cols].rolling(5, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols = ["w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
rolling_averages(group, cols, new_cols)

,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,...,target,w_ace_rolling,w_df_rolling,w_svpt_rolling,w_1stIn_rolling,w_1stWon_rolling,w_2ndWon_rolling,w_SvGms_rolling,w_bpSaved_rolling,w_bpFaced_rolling
tourney_id,,,,,,,,,,,,,,,,,,,,,
2006-2276,Zagreb,Carpet,32,A,20060130,26,104925,NaN,NaN,NovakDjokovic,...,1,5.6,2.2,59.8,36.0,26.8,12.6,8.8,3.0,4.2
2006-403,Miami Masters,Hard,96,M,20060320,16,104925,NaN,NaN,NovakDjokovic,...,1,6.8,1.6,53.6,32.6,25.2,12.0,8.4,2.4,3.2
2006-317,Amersfoort,Clay,32,A,20060717,5,104925,3,NaN,NovakDjokovic,...,1,8.2,1.6,58.4,36.2,29.2,12.8,9.6,2.4,3.2
2006-317,Amersfoort,Clay,32,A,20060717,19,104925,3,NaN,NovakDjokovic,...,1,6.4,1.6,53.0,32.6,25.8,11.8,9.0,1.6,2.4
2006-317,Amersfoort,Clay,32,A,20060717,26,104925,3,NaN,NovakDjokovic,...,1,8.2,2.0,52.2,33.4,27.0,11.0,8.8,1.4,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-580,Australian Open,Hard,128,G,20240115,164,104925,1,NaN,NovakDjokovic,...,1,13.0,2.0,88.6,56.4,44.4,19.6,15.0,2.4,3.6
2024-0404,Indian Wells Masters,Hard,128,M,20240304,269,104925,1,NaN,NovakDjokovic,...,1,13.8,1.8,103.2,65.4,51.6,22.6,17.2,3.4,5.0
2024-0410,Monte Carlo Masters,Clay,64,M,20240408,293,104925,1,NaN,NovakDjokovic,...,1,12.6,1.8,104.6,65.0,51.4,23.2,17.8,3.6,5.6


In [ ]:
matches_rolling = matches.groupby("winner_name").apply(lambda x: rolling_averages(x, cols, new_cols)) #applys the rolling averages function to each player

<ipython-input-56-0b125ae4f8e8>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("winner_name").apply(lambda x: rolling_averages(x, cols, new_cols)) #applys the rolling averages function to each player


In [ ]:
matches_rolling

tourney_name surface draw_size  \
winner_name     tourney_id                                           
AdrianMannarino 2011-580         Australian Open    Hard       128   
                2011-5012           Johannesburg    Hard        32   
                2011-5012           Johannesburg    Hard        32   
                2011-5012           Johannesburg    Hard        32   
                2011-402                 Memphis    Hard        32   
...                                          ...     ...       ...   
YounesElAynaoui 2010-451                    Doha    Hard        32   
YuichiSugita    2020-0891                   Pune    Hard        32   
ZhizhenZhang    2024-0451                   Doha    Hard        32   
                2024-0404   Indian Wells Masters    Hard       128   
                2024-0410    Monte Carlo Masters    Clay        64   

                           tourney_level tourney_date match_num winner_id  \
winner_name     tourney_id                                                  
AdrianMannarino 2011-580               G     20110117        35    105173   
                2011-5012              A     20110131         4    105173   
                2011-5012              A     20110131        18    105173   
                2011-5012              A     20110131        25    105173   
                2011-402               A     20110214         3    105173   
...                                  ...          ...       ...       ...   
YounesElAynaoui 2010-451               A     20100104        14    101962   
YuichiSugita    2020-0891              A     20200203       287    105216   
ZhizhenZhang    2024-0451              A     20240219       278    111190   
                2024-0404              M     20240304       235    111190   
                2024-0410              M     20240408       259    111190   

                           winner_seed winner_entry      winner_name  ...  \
winner_name     tourney_id                                            ...   
AdrianMannarino 2011-580           NaN          NaN  AdrianMannarino  ...   
                2011-5012            6          NaN  AdrianMannarino  ...   
                2011-5012            6          NaN  AdrianMannarino  ...   
                2011-5012            6          NaN  AdrianMannarino  ...   
                2011-402           NaN          NaN  AdrianMannarino  ...   
...                                ...          ...              ...  ...   
YounesElAynaoui 2010-451           NaN           WC  YounesElAynaoui  ...   
YuichiSugita    2020-0891            5          NaN     YuichiSugita  ...   
ZhizhenZhang    2024-0451          NaN          NaN     ZhizhenZhang  ...   
                2024-0404          NaN          NaN     ZhizhenZhang  ...   
                2024-0410          NaN          NaN     ZhizhenZhang  ...   

                           target w_ace_rolling w_df_rolling w_svpt_rolling  \
winner_name     tourney_id                                                    
AdrianMannarino 2011-580        1           4.0          0.6           58.4   
                2011-5012       1           4.2          0.8           63.4   
                2011-5012       1           4.4          1.4           65.4   
                2011-5012       1           4.6          2.2           71.4   
                2011-402        1           4.2          2.6           72.0   
...                           ...           ...          ...            ...   
YounesElAynaoui 2010-451        0          10.0          2.6           79.0   
YuichiSugita    2020-0891       1           7.6          3.8           94.2   
ZhizhenZhang    2024-0451       1           7.8          3.0           90.0   
                2024-0404       1           8.2          2.6           79.4   
                2024-0410       1           9.6          2.8           80.8   

                           w_1stIn_rolling w_1stWon_rolling w_2ndWon_rolling  \
winner_name  

In [ ]:
matches_rolling = matches_rolling.droplevel('winner_name') #drops the team name from the left col

In [ ]:
matches_rolling

,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,...,target,w_ace_rolling,w_df_rolling,w_svpt_rolling,w_1stIn_rolling,w_1stWon_rolling,w_2ndWon_rolling,w_SvGms_rolling,w_bpSaved_rolling,w_bpFaced_rolling
tourney_id,,,,,,,,,,,,,,,,,,,,,
2011-580,Australian Open,Hard,128,G,20110117,35,105173,NaN,NaN,AdrianMannarino,...,1,4.0,0.6,58.4,36.6,27.2,13.2,10.4,2.2,3.8
2011-5012,Johannesburg,Hard,32,A,20110131,4,105173,6,NaN,AdrianMannarino,...,1,4.2,0.8,63.4,39.8,30.4,14.0,11.4,1.6,3.2
2011-5012,Johannesburg,Hard,32,A,20110131,18,105173,6,NaN,AdrianMannarino,...,1,4.4,1.4,65.4,38.8,30.2,16.8,11.6,2.0,3.8
2011-5012,Johannesburg,Hard,32,A,20110131,25,105173,6,NaN,AdrianMannarino,...,1,4.6,2.2,71.4,41.2,31.4,18.4,12.6,2.0,4.0
2011-402,Memphis,Hard,32,A,20110214,3,105173,NaN,NaN,AdrianMannarino,...,1,4.2,2.6,72.0,41.6,32.4,18.8,12.4,1.8,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-451,Doha,Hard,32,A,20100104,14,101962,NaN,WC,YounesElAynaoui,...,0,10.0,2.6,79.0,50.6,39.4,14.0,12.2,3.6,4.8
2020-0891,Pune,Hard,32,A,20200203,287,105216,5,NaN,YuichiSugita,...,1,7.6,3.8,94.2,56.2,43.4,21.8,15.6,4.0,6.4
2024-0451,Doha,Hard,32,A,20240219,278,111190,NaN,NaN,ZhizhenZhang,...,1,7.8,3.0,90.0,57.2,42.6,17.6,14.6,3.2,5.2


In [ ]:
matches_rolling.index = range(matches_rolling.shape[0]) #resets the index and makes the indices unique

In [ ]:
matches_rolling

,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,...,target,w_ace_rolling,w_df_rolling,w_svpt_rolling,w_1stIn_rolling,w_1stWon_rolling,w_2ndWon_rolling,w_SvGms_rolling,w_bpSaved_rolling,w_bpFaced_rolling
0,Australian Open,Hard,128,G,20110117,35,105173,NaN,NaN,AdrianMannarino,...,1,4.0,0.6,58.4,36.6,27.2,13.2,10.4,2.2,3.8
1,Johannesburg,Hard,32,A,20110131,4,105173,6,NaN,AdrianMannarino,...,1,4.2,0.8,63.4,39.8,30.4,14.0,11.4,1.6,3.2
2,Johannesburg,Hard,32,A,20110131,18,105173,6,NaN,AdrianMannarino,...,1,4.4,1.4,65.4,38.8,30.2,16.8,11.6,2.0,3.8
3,Johannesburg,Hard,32,A,20110131,25,105173,6,NaN,AdrianMannarino,...,1,4.6,2.2,71.4,41.2,31.4,18.4,12.6,2.0,4.0
4,Memphis,Hard,32,A,20110214,3,105173,NaN,NaN,AdrianMannarino,...,1,4.2,2.6,72.0,41.6,32.4,18.8,12.4,1.8,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18577,Doha,Hard,32,A,20100104,14,101962,NaN,WC,YounesElAynaoui,...,0,10.0,2.6,79.0,50.6,39.4,14.0,12.2,3.6,4.8
18578,Pune,Hard,32,A,20200203,287,105216,5,NaN,YuichiSugita,...,1,7.6,3.8,94.2,56.2,43.4,21.8,15.6,4.0,6.4
18579,Doha,Hard,32,A,20240219,278,111190,NaN,NaN,ZhizhenZhang,...,1,7.8,3.0,90.0,57.2,42.6,17.6,14.6,3.2,5.2
18580,Indian Wells Masters,Hard,128,M,20240304,235,111190,NaN,NaN,ZhizhenZhang,...,1,8.2,2.6,79.4,51.6,38.8,14.8,13.0,3.6,5.2


In [ ]:
# This function trains a Random Forest model on historical data (pre-2022) and tests it on post-2022 data.
# It returns a DataFrame comparing actual vs. predicted target values and the precision score of the predictions.
# Arguments:
#   - data: DataFrame containing historical match data, including 'tourney_date' and target columns.
#   - predictors: List of column names to be used as predictors in the model.
# Returns:
#   - combined: DataFrame with columns for actual and predicted values on the test set.
#   - precision: Precision score indicating model performance on test data.

def make_predictions(data, predictors):
    train = data[data["tourney_date"] < '2022-01-01']
    test = data[data["tourney_date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
precision

0.8775193798449612

In [ ]:
combined

,actual,predicted
58,1,1
59,1,1
60,1,1
61,1,1
62,1,1
...,...,...
18527,1,1
18528,1,1
18579,1,1
18580,1,1


In [ ]:
combined = combined.merge(matches_rolling[["tourney_date", "winner_name", "loser_name", "winner_id"]], left_index=True, right_index=True) #merges combined df and matches_rolling by index, which will give more info when looking at combined

In [ ]:
combined

,actual,predicted,tourney_date,winner_name,loser_name,winner_id
58,1,1,20220117,AdrianMannarino,James Duckworth,105173
59,1,1,20220117,AdrianMannarino,Hubert Hurkacz,105173
60,1,1,20220117,AdrianMannarino,Aslan Karatsev,105173
61,1,1,20220131,AdrianMannarino,Alejandro Davidovich Fokina,105173
62,1,1,20220131,AdrianMannarino,David Goffin,105173
...,...,...,...,...,...,...
18527,1,1,20240219,YoshihitoNishioka,Flavio Cobolli,106415
18528,1,1,20240226,YoshihitoNishioka,Nuno Borges,106415
18579,1,1,20240219,ZhizhenZhang,Lorenzo Musetti,111190
18580,1,1,20240304,ZhizhenZhang,Aleksandar Kovacevic,111190
